In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_path = '/path/to/your/training-data'
val_path = 'path/to/your/validation-data'
test_path = 'path/to/your/testing-data'

batch_size = 16
img_height = 224
img_width = 224
input_shape = (img_height , img_width , 3)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.8, 1.2),
    horizontal_flip=True,
    vertical_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode='binary'
)

validation_generator = valid_datagen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode='binary'
)

# Print class indices
print(train_generator.class_indices)

In [ ]:
images, labels = next(train_generator)

def plot_augmented_images(images, labels, class_indices):
    class_names = list(class_indices.keys())
    plt.figure(figsize=(12, 12))
    for i in range(min(len(images), 9)):  # Plot up to 9 images
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        class_idx = np.argmax(labels[i])
        plt.title(class_names[class_idx])
        plt.axis('off')
    plt.show()

print(train_generator.class_indices)
plot_augmented_images(images, labels, train_generator.class_indices)

In [5]:
def resnet50():
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model_resnet50 = tf.keras.models.Model(inputs=base_model.input, outputs=pred)
    return model_resnet50

In [ ]:
model_resnet50 = resnet50()
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_resnet50.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model_resnet50.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=7, verbose=1, restore_best_weights=True)

history = model_resnet50.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=80,
    callbacks=[reduce_lr, early_stopping]
)

In [ ]:
plt.plot(history.history['loss'] , label = 'train_loss')
plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'] , label = 'train_acc')
plt.plot(history.history['val_accuracy'] , label = 'val_acc')
plt.legend()
plt.show()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary'
)

test_accu = model_resnet50.evaluate(test_generator)
print('The testing accuracy is :',test_accu[1]*100, '%')

In [ ]:
test_accuracy = test_accu[1]
test_loss = test_accu[0]

# Plotting Test Accuracy and Loss
plt.figure(figsize=(8, 4))

# Test Accuracy
plt.subplot(1, 2, 1)
plt.bar('Test Accuracy', test_accuracy, color='green', width=0.4)
plt.ylim(0, 1)
plt.title('Test Accuracy')
plt.ylabel('Accuracy')

# Test Loss
plt.subplot(1, 2, 2)
plt.bar('Test Loss', test_loss, color='red', width=0.4)
plt.title('Test Loss')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()

In [19]:
def inceptionresnetv2():
    base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model_inceptionresnetv2 = tf.keras.models.Model(inputs=base_model.input, outputs=pred)
    return model_inceptionresnetv2

In [ ]:
model_inceptionresnetv2 = inceptionresnetv2()
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_inceptionresnetv2.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model_inceptionresnetv2.summary()

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta = 0.0001, patience=7, verbose=1, restore_best_weights=True)

history = model_inceptionresnetv2.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=[reduce_lr, early_stopping])

In [ ]:
plt.plot(history.history['loss'] , label = 'train_loss')
plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'] , label = 'train_acc')
plt.plot(history.history['val_accuracy'] , label = 'val_acc')
plt.legend()
plt.show()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary'
)

test_accu = model_inceptionresnetv2.evaluate(test_generator)
print('The testing accuracy is :',test_accu[1]*100, '%')

In [ ]:
test_accuracy = test_accu[1]
test_loss = test_accu[0]

# Plotting Test Accuracy and Loss
plt.figure(figsize=(8, 4))

# Test Accuracy
plt.subplot(1, 2, 1)
plt.bar('Test Accuracy', test_accuracy, color='green', width=0.4)
plt.ylim(0, 1)
plt.title('Test Accuracy')
plt.ylabel('Accuracy')

# Test Loss
plt.subplot(1, 2, 2)
plt.bar('Test Loss', test_loss, color='red', width=0.4)
plt.title('Test Loss')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()

In [ ]:
def vgg19():
    base_model = tf.keras.applications.VGG19(weights = 'imagenet', include_top = False, input_shape=(224,224,3))
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model_vgg19 = tf.keras.models.Model(inputs=base_model.input, outputs=pred)
    return model_vgg19

model_vgg19 = vgg19()
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_vgg19.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model_vgg19.summary()

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta = 0.0001, patience=7, verbose=1, restore_best_weights=True)

history = model_vgg19.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=[reduce_lr, early_stopping])

In [ ]:
plt.plot(history.history['loss'] , label = 'train_loss')
plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'] , label = 'train_acc')
plt.plot(history.history['val_accuracy'] , label = 'val_acc')
plt.legend()
plt.show()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary'
)

test_accu = model_vgg19.evaluate(test_generator)
print('The testing accuracy is :',test_accu[1]*100, '%')

In [ ]:
test_accuracy = test_accu[1]
test_loss = test_accu[0]

# Plotting Test Accuracy and Loss
plt.figure(figsize=(8, 4))

# Test Accuracy
plt.subplot(1, 2, 1)
plt.bar('Test Accuracy', test_accuracy, color='green', width=0.4)
plt.ylim(0, 1)
plt.title('Test Accuracy')
plt.ylabel('Accuracy')

# Test Loss
plt.subplot(1, 2, 2)
plt.bar('Test Loss', test_loss, color='red', width=0.4)
plt.title('Test Loss')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()